### Sleep stage classification from Respiration signal.

Selection Task:

Use the UCI HAR dataset and build a model to predict various activity classes.
Use Deep Learning models like LSTM, and 1D Cnns for modeling.
How can the same be done using Machine Learning models like Random forest, SVM, and Logistic regression?
Make a .ipynb file to demonstrate modeling using Deep Learning.
Make a .ipynb file to demonstrate modeling using Machine learning.
YOU WILL BE ASKED TO EXPLAIN YOUR CODE.
Reach out to mentors when in need.


## Deep Learning Models

In [110]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,LSTM, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from keras.layers import Dense, GlobalAveragePooling1D, BatchNormalization, MaxPool1D, Reshape, Activation

#### Function to Normalization and one-hot encoding for y

In [111]:
# Assuming X is your data and y are your labels
def Normalization(X,y):
    X = np.array(X)
    y = np.array(y)
    y = y -1
    # Normalize the data (e.g., Min-Max scaling or Standardization)
    X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Min-Max scaling

    # If you need to convert your labels to one-hot encoding
    y = to_categorical(y)
    return X,y

#### Here we are using the time series data for the Deep Learning models as a input

* We are using the X_train from MakeDataset which was created in the Assignment-1 in Machine Learning course <br>
* Both 1D-CNN and LSTM expects the input to be #Dshape means: <br>
* [Batch size, time-steps, Features]<br>
* so X_train.shape = [126,500,3], in which 126 is the no of smaples and the 500 is the time steps we have (50hz for 10s data), 3 Features accx,accy,accz respectively

In [130]:
from HAR.MakeDataset import X_train, y_train, X_test, y_test
ACTIVITIES = {
    1: 'WALKING'            ,
    2: 'WALKING_UPSTAIRS'   ,
    3: 'WALKING_DOWNSTAIRS' ,
    4: 'SITTING'            ,
    5: 'STANDING'           ,
    6: 'LAYING'             ,
}
# X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]))
# X_test = X_test.reshape((X_test.shape[0], X_test.shape[1]*X_test.shape[2]))
y_train_n = y_train
y_test_n = y_test
X_train ,y_train = Normalization(X_train,y_train)
X_test, y_test = Normalization(X_test,y_test)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)


(126, 500, 3) (126, 6) (54, 500, 3) (54, 6)


## Implementaion of 1D-CNN

* 1D-CNN implementaion with 2 layers of 1D-CNN <br>
* In between this 2 layers 2 times of MaxPooling for the reduction of the size.
* Apprently we have 100 dense layers
* At last we have ouput layer with 6 ouputs for 6-activites and applied softmax activiation
* Then we are using adam optimizer and loss as categorical loss

In [113]:

n_runs = 10
acc = []
pre = []
recall = []
f1_scores = []

for i in range(n_runs):
    print(f'{i}th run')

    model = Sequential()
    model.add(Conv1D(filters=64,kernel_size=2,activation='relu',input_shape=(500,3)))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Conv1D(filters=64,kernel_size=2,activation='relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Flatten())
    model.add(Dense(100,activation='relu'))
    model.add(Dense(len(ACTIVITIES), activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(X_train,y_train,epochs=20,batch_size=32,validation_data=(X_test,y_test),verbose=0)

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred,axis=1)
    y_true = np.argmax(y_test,axis=1)
    acc.append(accuracy_score(y_true,y_pred_classes))
    pre.append(precision_score(y_true,y_pred_classes, average='weighted'))
    recall.append(recall_score(y_true,y_pred_classes, average='weighted'))
    f1_scores.append(f1_score(y_true,y_pred_classes, average='weighted'))



0th run
2/2 [==============================] - 0s 11ms/step
1th run
2/2 [==============================] - 0s 11ms/step
2th run


c:\Users\Shiva\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\Users\Shiva\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2/2 [==============================] - 0s 13ms/step
3th run
2/2 [==============================] - 0s 12ms/step
4th run
2/2 [==============================] - 0s 15ms/step
5th run
2/2 [==============================] - 0s 11ms/step
6th run
2/2 [==============================] - 0s 23ms/step
7th run
2/2 [==============================] - 0s 12ms/step
8th run
2/2 [==============================] - 0s 11ms/step
9th run
2/2 [==============================] - 0s 12ms/step


In [114]:
print(f'Average of all evalution metrics:')
print(f'Accuracy :{np.mean(acc)}')
print(f'Precision :{np.mean(pre)}')
print(f'Recall :{np.mean(recall)}')
print(f'F1 Score :{np.mean(f1_scores)}')

Average of all evalution metrics:
Accuracy :0.6055555555555556
Precision :0.5963051395771984
Recall :0.6055555555555556
F1 Score :0.5870995014819267


#### Implemetaion of the LSTM 

* In this also we have 2 layers of LSTM and then Dense and output as as CNN

In [115]:
n_runs = 1
acc_LSTM = []
pre_LSTM = []
recall_LSTM = []
f1_LSTM = []
for i in range(n_runs):
    print(f'{i}th run')

    model = Sequential()
    model.add(LSTM(32,return_sequences=True,input_shape=(500,3),activation='relu'))
    model.add(LSTM(32,activation= 'relu'))
    
    model.add(Dense(len(ACTIVITIES),activation= 'softmax'))
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred,axis=1)
    y_true = np.argmax(y_test,axis=1)
    acc_LSTM.append(accuracy_score(y_true,y_pred_classes))
    pre_LSTM.append(precision_score(y_true,y_pred_classes, average='weighted'))
    recall_LSTM.append(recall_score(y_true,y_pred_classes, average='weighted'))
    f1_LSTM.append(f1_score(y_true,y_pred_classes, average='weighted'))



0th run
Epoch 1/20
4/4 [==============================] - 6s 645ms/step - loss: 1.7899 - accuracy: 0.1667 - val_loss: 1.7870 - val_accuracy: 0.1667
Epoch 2/20
4/4 [==============================] - 2s 509ms/step - loss: 1.7834 - accuracy: 0.1667 - val_loss: 1.7813 - val_accuracy: 0.1667
Epoch 3/20
4/4 [==============================] - 2s 495ms/step - loss: 1.7768 - accuracy: 0.2222 - val_loss: 1.7746 - val_accuracy: 0.2593
Epoch 4/20
4/4 [==============================] - 2s 495ms/step - loss: 1.7693 - accuracy: 0.2857 - val_loss: 1.7667 - val_accuracy: 0.2963
Epoch 5/20
4/4 [==============================] - 2s 499ms/step - loss: 1.7605 - accuracy: 0.3016 - val_loss: 1.7556 - val_accuracy: 0.3333
Epoch 6/20
4/4 [==============================] - 2s 495ms/step - loss: 1.7454 - accuracy: 0.3095 - val_loss: 1.7389 - val_accuracy: 0.3333
Epoch 7/20
4/4 [==============================] - 2s 491ms/step - loss: 1.7262 - accuracy: 0.3254 - val_loss: 1.7144 - val_accuracy: 0.3333
Epoch 8/20
4

In [116]:
print(f'Average of all evalution metrics:')
print(f'Accuracy :{np.mean(acc_LSTM)}')
print(f'Precision :{np.mean(pre_LSTM)}')
print(f'Recall :{np.mean(recall_LSTM)}')
print(f'F1 Score :{np.mean(f1_LSTM)}')

Average of all evalution metrics:
Accuracy :0.16666666666666666
Precision :0.027777777777777776
Recall :0.16666666666666666
F1 Score :0.047619047619047616


In [117]:
print(np.isinf(X_train).any())
print(np.isinf(y_train).any())
print(np.isinf(X_test).any())
print(np.isinf(y_test).any())

False
False
False
False


###
* So 1D-CNN is working fine but LSTM not it is reaching loss = infinity after few iteration of the epochs.<br>
* So I changed the Learning rate to 0.01 to 0.0001 but still its going to the inifity

### Reading Material implemenation 

* This is the code where the combination of 1D-CNN and LSTM are used so I have used the code from the reading material

In [128]:
model_CL = Sequential()
model_CL.add(LSTM(32, return_sequences=True, input_shape=(500,3), activation='relu'))
model_CL.add(LSTM(32,return_sequences=True, activation='relu'))
model_CL.add(Reshape((1, 500, 32)))
model_CL.add(Conv1D(filters=64,kernel_size=2, activation='relu', strides=2))
model_CL.add(Reshape((250, 64)))
model_CL.add(MaxPool1D(pool_size=4, padding='same'))
model_CL.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
model_CL.add(Reshape((62, 192)))
model_CL.add(GlobalAveragePooling1D())
model_CL.add(BatchNormalization(epsilon=1e-06))
model_CL.add(Dense(6))
model_CL.add(Activation('softmax'))

print(model_CL.summary())

Model: "sequential_177"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_54 (LSTM)              (None, 1500, 32)          4352      
                                                                 
 lstm_55 (LSTM)              (None, 1500, 32)          8320      
                                                                 
 reshape_17 (Reshape)        (None, 1, 1500, 32)       0         
                                                                 
 conv1d_307 (Conv1D)         (None, 1, 750, 64)        4160      
                                                                 
 reshape_18 (Reshape)        (None, 750, 64)           0         
                                                                 
 max_pooling1d_302 (MaxPooli  (None, 188, 64)          0         
 ng1D)                                                           
                                                    

In [125]:
model_CL.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model_CL.fit(X_train,
                    y_train, 
                    batch_size= 192, 
                    epochs=100,
                    validation_data=(X_test, y_test)
                   )



Epoch 1/100


1/1 [==============================] - 9s 9s/step - loss: 1.8540 - accuracy: 0.1190 - val_loss: 1.7914 - val_accuracy: 0.1667
Epoch 2/100
1/1 [==============================] - 3s 3s/step - loss: 1.6919 - accuracy: 0.3254 - val_loss: 1.7913 - val_accuracy: 0.1852
Epoch 3/100
1/1 [==============================] - 3s 3s/step - loss: 1.6138 - accuracy: 0.3254 - val_loss: 1.7913 - val_accuracy: 0.1667
Epoch 4/100
1/1 [==============================] - 3s 3s/step - loss: 1.5753 - accuracy: 0.3254 - val_loss: 1.7912 - val_accuracy: 0.1667
Epoch 5/100
1/1 [==============================] - 3s 3s/step - loss: 1.5456 - accuracy: 0.3254 - val_loss: 1.7912 - val_accuracy: 0.1667
Epoch 6/100
1/1 [==============================] - 3s 3s/step - loss: 1.5284 - accuracy: 0.3254 - val_loss: 1.7912 - val_accuracy: 0.1667
Epoch 7/100
1/1 [==============================] - 3s 3s/step - loss: 1.5116 - accuracy: 0.3254 - val_loss: 1.7912 - val_accuracy: 0.1667
Epoch 8/100
1/1 [=============================

KeyboardInterrupt: 

In [120]:
y_pred = model_CL.predict(X_test)
y_pred_classes = np.argmax(y_pred,axis=1)
y_true = np.argmax(y_test,axis=1)

acc_CL = (accuracy_score(y_true,y_pred_classes))
pre_CL =(precision_score(y_true,y_pred_classes, average='weighted'))
recall_CL = (recall_score(y_true,y_pred_classes, average='weighted'))
f1_score_CL = (f1_score(y_true,y_pred_classes, average='weighted'))

print(f'Results of all evalution metrics:')
print(f'Accuracy :{acc_CL}')
print(f'Precision :{pre_CL}')
print(f'Recall :{recall_CL}')
print(f'F1 Score :{f1_score_CL}')

2/2 [==============================] - 1s 142ms/step
Results of all evalution metrics:
Accuracy :0.16666666666666666
Precision :0.027777777777777776
Recall :0.16666666666666666
F1 Score :0.047619047619047616


### Still the results where not better I will look in to it more deeper I am currently learning LSTM.